# Scrape the InfoBox for each link
### https://en.wikipedia.org/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft
### Store the InfoBoxes into a Dafaframe

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# Find the sections "1910s and 1920s" and "External Links" to minimize any uncessary links
start_section = soup.find("span", {"id": "1910s_and_1920s"})
end_section = soup.find("span", {"id": "External_links"})

# Empty list to store the URLs
urls = []

# Find all the UL tags before the "External links" section
ul_tags = start_section.find_next("ul")

while ul_tags and ul_tags.find_next("span") != end_section:
    # Find all the links inside the UL tag
    links = ul_tags.find_all("a")
    for link in links:
        # Get the URL of the link
        link_url = link.get("href")
        if link_url and link_url.startswith("/wiki/") and ":" not in link_url:
            # Append the URL to the list
            urls.append("https://en.wikipedia.org" + link_url)
    ul_tags = ul_tags.find_next("ul")
dfs = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    infoboxes = soup.find_all('table', {'class': 'infobox'})
    if infoboxes:
        for infobox in infoboxes:
            for df in pd.read_html(str(infobox)):
                dfs.append(df)
    else:
        print(f"No table found for {url}")

No table found for https://en.wikipedia.org/wiki/Martin_and_Osa_Johnson
No table found for https://en.wikipedia.org/wiki/Aeroelasticity#Flutter
No table found for https://en.wikipedia.org/wiki/Helen_E._Hokinson
No table found for https://en.wikipedia.org/wiki/Decoto,_California
No table found for https://en.wikipedia.org/wiki/Mid-air_collision
No table found for https://en.wikipedia.org/wiki/Mount_Takamagahara
No table found for https://en.wikipedia.org/wiki/Rahman_Dadman
No table found for https://en.wikipedia.org/wiki/2002_Africa_One_Antonov_An-26_crash
No table found for https://en.wikipedia.org/wiki/Polaris_Award
No table found for https://en.wikipedia.org/wiki/Takeoff


In [2]:
result_df = pd.concat(dfs)
result_df.reset_index
result_df.head()

c:\Users\extra\anaconda3\lib\site-packages\pandas\core\indexes\api.py:221: RuntimeWarning: '<' not supported between instances of 'tuple' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
c:\Users\extra\anaconda3\lib\site-packages\pandas\core\indexes\base.py:2955: RuntimeWarning: '<' not supported between instances of 'tuple' and 'int', sort order is undefined for incomparable objects
  return left.union(right, sort=sort)


,0,1,Accident,Accident.1,Bronson M. Cutting,Bronson M. Cutting.1,Carole Lombard,Carole Lombard.1,Cecilie of Greece and Denmark,Cecilie of Greece and Denmark.1,...,Osh International AirportОш эл аралык аэропорту,Osh International AirportОш эл аралык аэропорту.1,Video of the crash,Video of the crash.1,Moments of the crash,Moments of the crash.1,Mid-air collision,Mid-air collision.1,Raman Pratasevich,Raman Pratasevich.1
0,Wingfoot Air Express,Wingfoot Air Express,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Accident,Accident,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date,"July 21, 1919",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Summary,In-flight fire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Site,"Chicago, Illinois, United States .mw-parser-ou...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop the extra columns

In [3]:
df2 = result_df.drop(result_df.columns[2:],axis = 1)
df2

,0,1
0,Wingfoot Air Express,Wingfoot Air Express
1,Accident,Accident
2,Date,"July 21, 1919"
3,Summary,In-flight fire
4,Site,"Chicago, Illinois, United States .mw-parser-ou..."
...,...,...
20,Injuries,40
21,Survivors,108
22,Ground casualties,Ground casualties
23,Ground fatalities,2


### Drop the empty rows

In [4]:
empty = df2.dropna()
empty

,0,1
0,Wingfoot Air Express,Wingfoot Air Express
1,Accident,Accident
2,Date,"July 21, 1919"
3,Summary,In-flight fire
4,Site,"Chicago, Illinois, United States .mw-parser-ou..."
...,...,...
20,Injuries,40
21,Survivors,108
22,Ground casualties,Ground casualties
23,Ground fatalities,2


### Store the Dataframe into an excel file

In [5]:
empty.to_excel('scraped_data.xlsx')